# Power Outage

**Name(s)**: Luke, Andrew

**Website Link**: https://lukelin15.github.io/Power-Outage-Analysis/

# Power Outage Analysis

In this Jupyter Notebook, we will analyze a dataset containing major power outage data in the continental U.S. The dataset covers the period from January 2000 to July 2016.

## Import Statements

### Pandas
Pandas is a powerful data manipulation library. We will use it to load, clean, and analyze the power outage dataset.

### NumPy
NumPy is a library for numerical operations in Python. It provides support for large, multi-dimensional arrays and matrices. We may use it for numerical computations related to the power outage data.

### OS
The os module provides a way to interact with the operating system. We might use it for handling file paths or checking the existence of files.

### Folium
Folium is a Python library that makes it easy to visualize spatial data and create interactive maps. We will use it to create an interactive map displaying the geographical distribution of power outages.

### Geopy
Geopy provides tools for geocoding (finding the latitude and longitude of an address) and reverse geocoding (finding the address of a set of latitude and longitude coordinates). We may use it in conjunction with Folium for location-based analysis.

### HeatMap (Folium Plugin)
The HeatMap plugin from Folium allows us to create a heatmap layer on our interactive map. It can be used to visualize the intensity or concentration of power outages in different geographical areas.

In [1]:
import pandas as pd
import numpy as np
import os
from scipy.stats import ks_2samp

# Interactive Map 
import folium
from geopy.geocoders import Nominatim
from folium.plugins import HeatMap

import plotly.express as px
import plotly.graph_objects as go
pd.options.plotting.backend = 'plotly'

# Data Cleaning and Preparation

In this section, we perform several steps to clean and prepare the power outage data for analysis.

In [2]:
# Read the Excel file into a pandas DataFrame
outage = pd.read_excel("outage.xlsx", sheet_name="Masterdata")

# Drop informational rows
outage_cleaned = outage.drop(range(4)).dropna(axis=1, how='all')

# Set column names based on the first row
outage_cleaned.columns = outage_cleaned.iloc[0]

# Drop rows related to units and variables
outage_cleaned = outage_cleaned.drop([4, 5])
outage_cleaned = outage_cleaned.drop(columns="variables")

# Combine 'OUTAGE.START.DATE' and 'OUTAGE.START.TIME' into a new datetime column
outage_cleaned['OUTAGE.START'] = pd.to_datetime(outage_cleaned['OUTAGE.START.DATE']) + pd.to_timedelta(outage_cleaned['OUTAGE.START.TIME'].astype(str))

# Combine 'OUTAGE.RESTORATION.DATE' and 'OUTAGE.RESTORATION.TIME' into a new datetime column
outage_cleaned['OUTAGE.RESTORATION'] = pd.to_datetime(outage_cleaned['OUTAGE.RESTORATION.DATE']) + pd.to_timedelta(outage_cleaned['OUTAGE.RESTORATION.TIME'].astype(str))

# Drop the original date and time columns
outage_cleaned = outage_cleaned.drop(['OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME'], axis=1)

# Replace "NA" with NaN for missing values
outage_cleaned.replace("NA", np.nan, inplace=True)

#Essential Columns Analysis
essential_col = [
    "U.S._STATE", 
    'OUTAGE.START',
    "OUTAGE.RESTORATION", 
    "CUSTOMERS.AFFECTED", 
    "OUTAGE.DURATION", 
    "DEMAND.LOSS.MW", 
    "RES.PRICE", 
    "NERC.REGION", 
    "CLIMATE.REGION",
    "CAUSE.CATEGORY",
    "TOTAL.SALES"
]
# Display the cleaned DataFrame
outage_cleaned = outage_cleaned[essential_col]
print(outage_cleaned.shape)


(1534, 11)


# Data Visualization
The provided code utilizes the plotly libraries to create histograms for four key columns in the power outage dataset: 'OUTAGE.DURATION', 'DEMAND.LOSS.MW', 'CUSTOMERS.AFFECTED', and 'RES.PRICE'. Below are some objectives we are attempting to find by illustrating these graphs. 

## Distribution of OUTAGE.DURATION

Objective:
- Understand the distribution of outage durations to identify common patterns and outliers.

Justification:

- Outage duration is a crucial metric, providing insights into the temporal aspect of power outages.
- A histogram with a kernel density estimate (KDE) allows us to observe the central tendency and spread of outage durations.

## Distribution of DEMAND.LOSS.MW

Objective:
- Analyze the distribution of demand loss in terms of megawatts during power outages.

Justification:

- 'DEMAND.LOSS.MW' represents the amount of demand lost during an outage, offering insights into the severity of power disruptions.
- A histogram with KDE enables visualization of the range and frequency of demand losses.

## Distribution of CUSTOMERS.AFFECTED

Objective:
- Explore the impact of power outages on the number of customers affected.

Justification:
- 'CUSTOMERS.AFFECTED' provides information on the scale of the outage in terms of affected users.
- A histogram with KDE helps understand the distribution of the number of affected customers.

## Distribution of RES.PRICE

Objective:
- Investigate the distribution of residential electricity prices during power outage events.

Justification:
- 'RES.PRICE' represents the residential electricity price, which may correlate with the severity or causes of power outages.
- A histogram with KDE allows us to visualize the distribution of residential electricity prices.

## Summary:
These visualizations serve as essential exploratory tools to gain a deeper understanding of key features in the power outage dataset. By analyzing the distributions of relevant columns, we can identify trends, outliers, and potential relationships between variables, providing a foundation for further analysis and hypothesis formulation.

In [3]:
# Plot distribution of OUTAGE.DURATION
fig1 = go.Figure(data=[go.Histogram(x=outage_cleaned['OUTAGE.DURATION'], nbinsx=30)])
fig1.update_layout(title_text='Distribution of OUTAGE.DURATION', xaxis_title='Duration (minutes)', yaxis_title='Frequency')
fig1.show()

# Plot distribution of DEMAND.LOSS.MW
fig2 = go.Figure(data=[go.Histogram(x=outage_cleaned['DEMAND.LOSS.MW'], nbinsx=30)])
fig2.update_layout(title_text='Distribution of DEMAND.LOSS.MW', xaxis_title='Demand Loss (Megawatt)', yaxis_title='Frequency')
fig2.show()

# Plot distribution of CUSTOMERS.AFFECTED
fig3 = go.Figure(data=[go.Histogram(x=outage_cleaned['CUSTOMERS.AFFECTED'], nbinsx=30)])
fig3.update_layout(title_text='Distribution of CUSTOMERS.AFFECTED', xaxis_title='Number of Customers Affected', yaxis_title='Frequency')
fig3.show()

# Plot distribution of RES.PRICE
fig4 = go.Figure(data=[go.Histogram(x=outage_cleaned['RES.PRICE'], nbinsx=30)])
fig4.update_layout(title_text='Distribution of RES.PRICE', xaxis_title='Residential Price (cents/kWh)', yaxis_title='Frequency')
fig4.show()

# Exploratory Data Analysis (EDA) Visualizations

## Uncovering Patterns and Comparisons

In this section, we delve into visualizations aimed at uncovering patterns and making insightful comparisons within the power outage dataset. Utilizing scatter plots and box plots, we explore relationships between outage duration and affected customers, as well as variations in customer impact across different NERC regions. These visualizations serve as a foundational step in understanding the nuances and potential factors influencing power outage events.

### Scatter Plot of OUTAGE.DURATION vs. CUSTOMERS.AFFECTED
**Description:**
This scatter plot is created to visually assess the potential correlation or pattern between the duration of power outages ('OUTAGE.DURATION') and the number of customers affected ('CUSTOMERS.AFFECTED'). Each point on the plot represents a specific power outage event, with the x-axis indicating the duration of the outage in minutes and the y-axis representing the number of affected customers. By examining the distribution of points, one can infer whether longer durations tend to result in a higher number of affected customers or if there are other patterns worth exploring.

### Scatter Plot of OUTAGE.RESTORATION vs. CUSTOMERS.AFFECTED
**Description:**
This scatter plot provides a visual representation of the relationship between the restoration time of power outages ('OUTAGE.RESTORATION') and the number of customers affected ('CUSTOMERS.AFFECTED'). Each point on the plot corresponds to a specific power outage event, with the x-axis depicting the restoration time and the y-axis representing the number of affected customers. The plot allows for an exploration of patterns and trends, enabling observations about whether quicker restoration times correlate with fewer affected customers or if there are notable outliers indicating unusual scenarios. The tooltips include information about the state and cause category for each data point, providing additional context for the analysis.

### Box Plot of NERC.REGION vs. CUSTOMERS.AFFECTED
**Description:**
This box plot is designed to compare the distribution of the number of customers affected ('CUSTOMERS.AFFECTED') across different NERC (North American Electric Reliability Corporation) regions. Each box represents the interquartile range (IQR) of the data for a specific region, with the median indicated by the horizontal line inside the box. Outliers are also displayed as individual points. By examining these box plots, one can gain insights into the variations in the impact of power outages on customers across different NERC regions.

In [4]:
# Create a scatter plot
fig1 = px.scatter(outage_cleaned, x='OUTAGE.DURATION', y='CUSTOMERS.AFFECTED', title='Scatter Plot of OUTAGE.DURATION vs. CUSTOMERS.AFFECTED')
fig1.update_xaxes(title_text='OUTAGE.DURATION (minutes)')
fig1.update_yaxes(title_text='CUSTOMERS.AFFECTED')
fig1.show()
fig1.write_html(os.path.join('Assets', 'scatter.html'), include_plotlyjs='cdn')

# Create a scatter plot using Plotly Express
outage_cleaned['OUTAGE.RESTORATION'] = pd.to_datetime(outage_cleaned['OUTAGE.RESTORATION'])
fig2 = px.scatter(outage_cleaned, x='OUTAGE.RESTORATION', y='CUSTOMERS.AFFECTED', title='Bivariate Analysis: OUTAGE.RESTORATION vs CUSTOMERS.AFFECTED',
                 labels={'OUTAGE.RESTORATION': 'Outage Restoration', 'CUSTOMERS.AFFECTED': 'Customers Affected'},
                 hover_data=['U.S._STATE', 'CAUSE.CATEGORY'])

# Show the plot
fig2.show()

# Create a box plot
fig3 = px.box(outage_cleaned, x='NERC.REGION', y='CUSTOMERS.AFFECTED', title='Box Plot of NERC.REGION vs. CUSTOMERS.AFFECTED')
fig3.update_xaxes(title_text='NERC.REGION')
fig3.update_yaxes(title_text='CUSTOMERS.AFFECTED')
fig3.show()
fig2.write_html(os.path.join('Assets', 'box_plot.html'), include_plotlyjs='cdn')


# Customer Impact Analysis

## Aggregating Customer Impact Data (Groupby)
Description:
In this section, the code groupes the data by 'State' and calculates the average number of customers affected ('CUSTOMERS.AFFECTED') for each state. This provides insights into the regional variations in the impact of power outages. The result is displayed in a DataFrame, showcasing the average customer impact for each state.

## Aggregating Customer Impact Data (Pivot Table)
Description:
In this section, the code groupes the data by 'State' and calculates the average number of customers affected ('CUSTOMERS.AFFECTED') for each state. This provides insights into the regional variations in the impact of power outages. The result is displayed in a DataFrame, showcasing the average customer impact for each state.


In [5]:
# Group by 'State' and calculate the average 'CUSTOMERS.AFFECTED' for each year
average_customers_by_state= outage_cleaned.groupby("U.S._STATE")["CUSTOMERS.AFFECTED"].mean()
average_customers_by_state = average_customers_by_state.to_frame().reset_index()

# Display the result
print("Average Customers Affected by State:")
average_customers_by_state

Average Customers Affected by State:


,U.S._STATE,CUSTOMERS.AFFECTED
0,Alabama,94328.800000
1,Alaska,14273.000000
2,Arizona,64402.666667
3,Arkansas,47673.846154
4,California,201365.716535
5,Colorado,41060.636364
6,Connecticut,60339.230769
7,Delaware,3475.000000
8,District of Columbia,194709.222222
9,Florida,289369.090909


In [6]:
# Pivot the data to examine average 'CUSTOMERS.AFFECTED' by 'CLIMATE.REGION' and 'CAUSE.CATEGORY'
pivot_table = outage_cleaned.pivot_table(values='CUSTOMERS.AFFECTED', index='CLIMATE.REGION', columns='CAUSE.CATEGORY', aggfunc='mean')

# Display the pivot table
print("\nAverage Customers Affected by Climate Region and Cause Category:")
pivot_table


Average Customers Affected by Climate Region and Cause Category:


CAUSE.CATEGORY,equipment failure,fuel supply emergency,intentional attack,islanding,public appeal,severe weather,system operability disruption
CLIMATE.REGION,,,,,,,
Central,87750.000000,0.0,110.714286,9666.666667,NaN,148706.653226,210450.000000
East North Central,NaN,NaN,660.111111,0.000000,7600.000000,134972.500000,759737.666667
Northeast,28575.750000,0.5,1055.580247,0.000000,18600.000000,169466.934524,530758.857143
Northwest,46651.500000,NaN,92.592593,0.000000,4000.000000,169284.034483,35000.000000
South,62721.666667,NaN,1042.833333,14500.000000,4917.636364,223231.144231,227102.047619
Southeast,145420.200000,NaN,0.000000,NaN,0.000000,206523.448276,75555.533333
Southwest,55666.666667,0.0,327.423077,35230.000000,NaN,85138.428571,135655.571429
West,198608.142857,0.0,14060.000000,5039.192308,NaN,361041.403509,152040.454545
West North Central,NaN,NaN,0.000000,NaN,34500.000000,74178.000000,NaN


# Enhancing Exploration with Folium: Geospatial Analysis
In this section, we aim to enhance the exploration of power outage effects by incorporating interactive geospatial analysis using Folium. The provided code includes a function to retrieve latitude and longitude information for each U.S. state. By applying this function to the 'U.S._STATE' column, the dataset is enriched with geographical coordinates. Rows with missing latitude and longitude information are then removed to ensure accuracy in the geospatial analysis.

Now that the latitude and longitude information is available, we can proceed to create interactive Folium maps to visualize and analyze the effects of power outages by state.

In [7]:
# Initialize the geolocator
geolocator = Nominatim(user_agent="my_geocoder")

# Function to get latitude and longitude
def get_lat_lon(location):
    try:
        location = geolocator.geocode(location)
        return (location.latitude, location.longitude)
    except:
        return None

# Apply the function to the 'U.S._STATE' column
average_customers_by_state['Latitude, Longitude'] = average_customers_by_state['U.S._STATE'].apply(get_lat_lon)

# Split the 'Latitude, Longitude' column into separate 'Latitude' and 'Longitude' columns
average_customers_by_state[['Latitude', 'Longitude']] = pd.DataFrame(average_customers_by_state['Latitude, Longitude'].tolist(), index=average_customers_by_state.index)

# Drop the 'Latitude, Longitude' column
average_customers_by_state = average_customers_by_state.drop('Latitude, Longitude', axis=1)

# Display the resulting DataFrame
average_customers_by_state

,U.S._STATE,CUSTOMERS.AFFECTED,Latitude,Longitude
0,Alabama,94328.800000,33.258882,-86.829534
1,Alaska,14273.000000,64.445961,-149.680909
2,Arizona,64402.666667,34.395342,-111.763275
3,Arkansas,47673.846154,35.204888,-92.447911
4,California,201365.716535,36.701463,-118.755997
5,Colorado,41060.636364,38.725178,-105.607716
6,Connecticut,60339.230769,41.650020,-72.734216
7,Delaware,3475.000000,38.692045,-75.401331
8,District of Columbia,194709.222222,38.893847,-76.988043
9,Florida,289369.090909,27.756767,-81.463983


# Visualizing Customer Impact with Folium Heatmap
This code snippet utilizes Folium to create an interactive heatmap visualizing the impact of power outages based on the number of affected customers. The map is centered on Minnesota, serving as the default location due to the order of the Excel file. The heatmap layer is constructed using latitude, longitude, and the 'CUSTOMERS.AFFECTED' column. Each point on the map contributes to the heatmap intensity, providing a spatial representation of the customer impact. 

In [8]:
# Remove Nan entries
average_customers_by_state = average_customers_by_state[~average_customers_by_state["CUSTOMERS.AFFECTED"].isna()]

# Create a Map instance
m = folium.Map(location=[46.7296, -94.6859], zoom_start=6)  # Location coordinates for Minnesota

# Prepare data for the heatmap
data = average_customers_by_state[['Latitude', 'Longitude', 'CUSTOMERS.AFFECTED']].values.tolist()

# Add the heatmap to the map
HeatMap(data).add_to(m)

# Display the map
m

# Average Outage Duration per State
In this section, we’re interested in understanding how the duration of power outages varies across different states. To do this, we first calculate the average outage duration for each state using the groupby function in pandas. This gives us a new DataFrame, avg_outage_duration, where each row corresponds to a state and the ‘OUTAGE.DURATION’ column contains the average outage duration for that state.

Next, we create a bar plot using Plotly Express (px.bar). The x-axis of the plot represents the different states, and the y-axis represents the average outage duration. Each bar in the plot corresponds to a state. The height of the bar represents the average outage duration for that state.

By visualizing the data in this way, we can easily compare the average outage duration across different states.

In [9]:
# Calculate the average outage duration per state
avg_outage_duration = outage_cleaned.groupby('U.S._STATE')['OUTAGE.DURATION'].mean().reset_index()

# Create a bar plot
fig = px.bar(avg_outage_duration, x='U.S._STATE', y='OUTAGE.DURATION',
             labels={'U.S._STATE':'State', 'OUTAGE.DURATION':'Average Outage Duration'},
             title='Average Outage Duration per State')

# Show the plot
fig.show()
fig.write_html(os.path.join('Assets', 'avg_outage_duration.html'), include_plotlyjs='cdn')


### Assessment of Missingness

# NMAR Analysis

We believe the column CAUSE.CATEGORY.DETAIL is likely to be NMAR as the column revolves around a detailed description of the event categories, and too complex of a description may cause nothing to be marked down instead. Possible data that could help make it MAR would be the uniqueness or complexity of the cause since more complex causes may not be easily inputed into the data.



# Missingness Dependency



In [10]:
def ks_query(missing, dependent):
    mar = outage_cleaned.copy()
    mar[dependent] = mar[dependent].astype(float)
    mar['missing'] = mar[missing].isna()
    res = ks_2samp(mar.query('missing')[dependent], mar.query('not missing')[dependent])
    return res

dur_vs_cust = ks_query('OUTAGE.DURATION', 'CUSTOMERS.AFFECTED')
dur_vs_sales = ks_query('OUTAGE.DURATION', 'TOTAL.SALES')

print(dur_vs_cust, dur_vs_sales)

duration_missing = outage_cleaned.copy()
duration_missing['missing'] = duration_missing['OUTAGE.DURATION'].isna()

fig1 = px.histogram(duration_missing, x='CUSTOMERS.AFFECTED', color='missing', histnorm='probability', marginal='box',
             title="customers affected by missingness of outage duration", barmode='overlay', opacity=0.7)
fig1.write_html(r'missing1.html', include_plotlyjs='cdn')

fig2 = px.histogram(duration_missing, x='TOTAL.SALES', color='missing', histnorm='probability', marginal='box',
             title="total sales by missingness of outage duration", barmode='overlay', opacity=0.7)
fig2.show()
fig2.write_html(r'missing2.html', include_plotlyjs='cdn')


KstestResult(statistic=0.12307261003644519, pvalue=0.3376185403211189, statistic_location=430000.0, statistic_sign=-1) KstestResult(statistic=0.21325109802822168, pvalue=0.010450324966936536, statistic_location=19970292.0, statistic_sign=-1)


### Hypothesis Testing

# Permutation Test for Mean Outage Duration (2005 vs. 2015)

In this section, we perform a permutation test to assess whether there is a significant difference in the mean outage duration between the years 2005 and 2015.

## Hypotheses

- **Null Hypothesis (H0):** There is no difference in mean outage duration for outages that occured in 2005 and in 2015.
- **Alternative Hypothesis (H1):** The mean outage duration for outages that occured in 2005 is larger than those that occured in 2015.

## Test Statistic

We choose the difference in mean outage duration between the two years as our test statistic. The observed difference is calculated using the actual data, and we compare this against a distribution of differences obtained by shuffling the outage duration data.

## Significance Level

We set the significance level at 0.05, which is a common choice in hypothesis testing.

## Permutation Test Procedure

1. **Data Preparation:**
    - We filter the dataset to include only rows from the years 2005 and 2015.
    - Missing values are dropped from the dataset.

2. **Observation:**
    - The observed difference in mean outage duration between 2005 and 2015 is computed.

3. **Permutation Test Loop (500 Repetitions):**
    - In each iteration, the outage duration data is shuffled, and the difference in mean outage duration is calculated.
    - These differences form a distribution under the null hypothesis.

4. **P-value Calculation:**
    - The proportion of permuted mean differences that are greater than or equal to the observed difference is calculated.

## Results

The resulting p-value is the probability of observing a difference in mean outage duration as extreme as the observed difference under the assumption that there is no true difference between the years 2005 and 2015.

## Conclusion

- **P-value:** 0.0
- **Conclusion:** If the p-value is less than the chosen significance level (0.05), we reject the null hypothesis in favor of the alternative hypothesis, suggesting a significant difference in mean outage duration between the years 2005 and 2015. From our tests, we have sufficient evidence to reject our null hypothesis. We therefore reject the idea that outage durations have stayed the same over the past 10 years.

## Justification

- The choice of a permutation test is appropriate when the assumptions of parametric tests are not met, or the distribution of the data is unknown.
- The test statistic (difference in means) aligns with the question of interest, comparing the average outage duration between the two years.
- A significance level of 0.05 is commonly used and provides a balance between Type I and Type II errors.

In [11]:
df = outage_cleaned.copy()
n_repetitions = 500

df['OUTAGE.DURATION'] = df['OUTAGE.DURATION'].astype(float)

shuffled = df[(df['OUTAGE.START'].dt.year == 2005) | (df['OUTAGE.START'].dt.year == 2015)]
shuffled['old_year'] = shuffled['OUTAGE.START'].dt.year == 2005


observed_difference = shuffled.groupby('old_year')['OUTAGE.DURATION'].mean().diff().iloc[-1]


differences = []
for _ in range(n_repetitions):
    
    with_shuffled = shuffled.assign(Shuffled_Duration=np.random.permutation(shuffled['OUTAGE.DURATION']))

    group_means = (
        with_shuffled
        .groupby('old_year')
        .mean()
        .loc[:, 'Shuffled_Duration']
    )
    difference = group_means.diff().iloc[-1]
    
    differences.append(difference)

p = (np.array(differences) >= observed_difference).mean()
print(differences)
fig = px.histogram(x=differences, histnorm='probability', marginal='box',
             title="Empirical Distribution of the Mean Difference", barmode='overlay', opacity=0.7)
fig.add_vline(x=observed_difference, line_width=3, line_dash="dash")
fig.show()
fig.write_html(os.path.join('Assets', 'conclusion.html'), include_plotlyjs='cdn')

C:\Users\lukel\AppData\Local\Temp\ipykernel_40616\591779296.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



[1418.7967260298615, -1140.5830115830117, -255.88817663817645, -164.04910955207788, -848.2872818852311, 424.2827635327635, -770.7599715099716, 62.754272351141935, -352.18250749426943, 816.5573025856047, 161.98430611885033, -177.47454545454548, -843.1858578733911, -53.77991452991455, 646.068946497518, -58.929090909090974, 1493.577679720537, 305.27991452991455, -390.35686707115246, -1455.9434523809525, -174.84241770102517, 349.3381903220002, 842.7249162405219, 1290.0696819268246, -926.9555065269351, 766.3218181818183, -498.8226495726499, -362.7257834757834, 429.4154437456323, -10.740081114441637, 1307.3311965811965, 0.18551204265486376, 213.34250764525996, 1136.8527272727274, 1047.017629069977, -1348.9945454545455, 853.9757150566652, 202.94727272727278, 945.1575822989748, -650.3763636363635, -634.8724030152603, 218.19090909090892, 830.409090909091, -536.1654545454544, -336.56934700485726, -306.83738082388936, -328.2243589743589, 750.2939878654165, 900.7323223417388, 350.05999999999995, 1